In [1]:
# Specify your target directory
# Traverse directory
# parse files to dict structure
# Compute derivative values
# Print medians

In [5]:
import os
from os.path import dirname, join
from glob import glob 
from statistics import median

In [6]:
def print_median(name, label, key, contents):
    print(name, label, "{:0.2%}".format(median([contents[d][key] for d in contents.keys()])))
    
def print_average(name, label, key, contents):
    avg = 0
    total_length = 0
    genome_length = sum([v["Ref Chr Total Size (No N's)"] for k,v in contents.items()])
    for chr in contents.keys():
        avg += contents[chr][key] * contents[chr]["Ref Chr Total Size (No N's)"] / genome_length
    print(name, label, "{:0.2%}".format(avg))

def summarize_stats(name, contents):
#     print("Median Chromosome Value")
#     print_median(name, 'identity', 'Ref Identity within Alignment', contents)
#     print_median(name, 'coverage', 'Alignment Coverage of Ref Chr', contents)
#     print_median(name, 'query coverage', "Alignment Coverage of Query Main Chr", contents)
    
    print_average(name, 'identity', 'Ref Identity within Alignment', contents)
    print_average(name, 'coverage', 'Alignment Coverage of Ref Chr', contents)
#     print_average(name, 'query coverage', "Alignment Coverage of Query Main Chr", contents)

In [7]:
def number(string):
    try:
        return int(string)
    except ValueError:
        return float(string)
    
def collect_stats(folder, prefix=''):
    contents = {}
      # Specify you target directory
    os.chdir(folder)
    # Traverse directory
    subdirs = glob(prefix + '*')
    for d in subdirs:
        stats_file = os.path.join(d, r'stats.txt')
        if os.path.exists(stats_file):
            with open(stats_file) as stat:
                # parse files to dict structure
                lines = [line.split('\t') for line in stat.read().splitlines()]
                v = {line[0]: number(line[1]) for line in lines if len(line) == 2}
                # Compute derivative values
                v["Total alignment Length"] = v['Aligned Variance in bp'] + v['Shared seq bp']
                v["Ref Chr Total Size (No N's)"] = v['Ref unique bp'] + v["Total alignment Length"]
                v["Ref Identity within Alignment"] = 1 - (v['Aligned Variance in bp'] / v['Total alignment Length'])
                v["Alignment Coverage of Ref Chr"] = v["Total alignment Length"] / v["Ref Chr Total Size (No N's)"]
                v["Alignment Coverage of Query Main Chr"] = v['Total alignment Length'] / (v['Total alignment Length'] + v["Query unique bp"])
                contents[d] = v
    # Print medians
    name = prefix or os.path.basename(folder)
#     print("Averages Weighted by Chromosome Length")
    summarize_stats(name, contents)
    return contents

In [10]:
root = r'E:\Projects\FluentDNA-2.4.1\www-data\dnadata'
# folder = os.path.join(root, )
genome = collect_stats(root, prefix='Human Hg38 vs Chimpanzee PanTro6_')
for name, stats in genome.items():
    print(f"\n\n{name[33:]}")
    for k,v in stats.items():
        print(f'{k}\t{v}')

Human Hg38 vs Chimpanzee PanTro6_ identity 98.65%
Human Hg38 vs Chimpanzee PanTro6_ coverage 94.95%


chr1
Ref Number of Gaps (all)	165609
Ref Gaps larger than 10bp	18763
Query Number of Gaps (all)	171053
Query Gaps larger than 10bp	21270
Ref Gaps larger than 100bp	4479
Query Gaps larger than 100bp	4832
Ref Gaps larger than 1000bp	1655
Query Gaps larger than 1000bp	1943
Ref N to query bp	18475471
Ref unique bp	8720120
Shared seq bp	220917707
Aligned Variance in bp	2830836
Query unique bp	8724657
Query N to ref in bp	2495044
Total alignment Length	223748543
Ref Chr Total Size (No N's)	232468663
Ref Identity within Alignment	0.9873481366088717
Alignment Coverage of Ref Chr	0.9624890516964001
Alignment Coverage of Query Main Chr	0.9624702675405165


chr10
Query Number of Gaps (all)	98327
Query Gaps larger than 10bp	12245
Ref Number of Gaps (all)	95484
Ref Gaps larger than 10bp	11050
Query Gaps larger than 100bp	2354
Query Gaps larger than 1000bp	620
Ref Gaps larger than 100bp	2504
Ref Gap

In [19]:
chr_order = 'chr1 chr2 chr3 chr4 chr5 chr6 chr7 chr8 chr9 chr10 chr11 chr12 chr13 chr14 chr15 chr16 chr17 chr18 chr19 chr20 chr21 chr22 chrX chrY'.split()
prefix = 'Human Hg38 vs Chimpanzee PanTro6_'
labels = list(genome['Human Hg38 vs Chimpanzee PanTro6_chr1'].keys())
print(labels)
print('\t'.join(chr_order))
for label in labels:
    if label:
        print(label, end='\t')
        for c in chr_order:
            
            v = genome[prefix + c][label] if label in genome[prefix + c] else 0
            print(v, end='\t')
        print()


['Ref Number of Gaps (all)', 'Ref Gaps larger than 10bp', 'Query Number of Gaps (all)', 'Query Gaps larger than 10bp', 'Ref Gaps larger than 100bp', 'Query Gaps larger than 100bp', 'Ref Gaps larger than 1000bp', 'Query Gaps larger than 1000bp', 'Ref N to query bp', 'Ref unique bp', 'Shared seq bp', 'Aligned Variance in bp', 'Query unique bp', 'Query N to ref in bp', 'Total alignment Length', "Ref Chr Total Size (No N's)", 'Ref Identity within Alignment', 'Alignment Coverage of Ref Chr', 'Alignment Coverage of Query Main Chr']
chr1	chr2	chr3	chr4	chr5	chr6	chr7	chr8	chr9	chr10	chr11	chr12	chr13	chr14	chr15	chr16	chr17	chr18	chr19	chr20	chr21	chr22	chrX	chrY
Ref Number of Gaps (all)	165609	170672	137201	139495	127304	123905	119253	103879	90319	95484	92132	96419	71882	65255	63744	66458	64312	55079	53722	45779	33180	33888	89016	35422	
Ref Gaps larger than 10bp	18763	18015	13383	13788	12607	12484	14143	11382	10275	11050	9946	9907	7416	6801	6843	9996	8164	5861	7683	5898	4281	5000	10685	7320	

In [8]:
all_results = {}
for comparison in ['hg38toPantro6']:
    all_results[comparison] = collect_stats(comparison)
    print()

hg38toPantro6 identity 98.67%
hg38toPantro6 coverage 91.46%



In [26]:
centromeres = {}
lines = open(r"E:\Google Drive\Research\Thesis - Genome Symmetry\Genome Symmetry Thesis Manuscript\Actively Used Figures\3\Unique Human Centromere Locations.txt", 'r').readlines()
for line_i in range(0, len(lines), 2):
    label1, start = lines[line_i].split('\t')
    label2, end = lines[line_i+1].split('\t')
    start, end = int(start), int(end)
    assert label1 == label2 and start < end, (label1, label2, start, end)
    centromeres[label1] = end - start
for c in chr_order:
    print(centromeres[c], end='\t')

3370400	2076300	3157700	2077200	4186600	1549700	4162600	2802300	2600400	2862200	4463500	3252100	2049700	2306200	2779400	2481000	4171000	5434500	3408800	3163400	2244000	2516800	4574000	662700	

# Stats for Syntenic versus Translocation from Chain file
We want to know how much of the alignment is a syntenic transfer 1:1 and how many bp have been rearranged between the two genomes.

In [1]:
from DDV.ChainFiles import chain_file_to_list, fetch_all_chains, Chain, ChainEntry

In [35]:
contig_names = 'chr1 chr2 chr3 chr4 chr5 chr6 chr7 chr8 chr9 chr10 chr11 chr12 chr13 chr14 chr15 chr16 chr17 chr18 chr19 chr20 chr21 chr22 chrX chrY'.split()
chain_stats = {name: {} for name in contig_names}
chain_path = r"D:\Genomes\Human\alignments\hg38ToPanTro6.over.chain"
headers = chain_file_to_list(chain_path, contig_names)  # just read the headers and discard the rest
len(headers)

28612

Chains are already sorted by highest score first.

In [36]:
from DNASkittleUtils.DDVUtils import pp

In [39]:
pp(headers[0].score), pp(headers[0].tSize), pp(headers[0].qSize)

('11,079,226,391', '242,193,529', '128,755,405')

In [8]:
pp(sum([e.size for e in headers[0].entries]))

'124,837,690'

The actual coverage tends to be slightly less than the list "query Size" which includes gaps. It appears chain Score is not bp at all.  Likely this is a z-score.

Sizes include difference already summed in.  I only know the % identity or total number of shared bp.  I don't know how that divides between my 3 categories, though if I assume it was evenly distributed, you could estimate shared bp in each category.  If I went through the unique sequence after having collapsed all ref gaps, then I'd have the exact number, since differences need not be attributed to a particular category.  We can switch to the translocations + non_syntenic + primary_chain that I've already calculated when we want to include differences.  "No Coverage" is already known.  
**Conclusion**: These stats are all coverage, not shared bp or identity.

In [63]:
chr2_chains = fetch_all_chains('chr2', 'chr2B', None, headers) +\
            fetch_all_chains('chr2', 'chr2A', None, headers)  # special case chr2
chr2_chains.sort(key=lambda chain: -chain.score)

for chromosome in contig_names:
    relevant = fetch_all_chains(chromosome, chromosome, None, headers)
    if chromosome == 'chr2':
        relevant = chr2_chains
    chain_stats[chromosome]['Primary Chain'] = sum([e.size for e in relevant[0].entries])
    non_syntenic = sum([e.size for chain in relevant[1:] for e in chain.entries]) if len(relevant) > 1 else 0
    chain_stats[chromosome]['Non-syntenic, Same Chromosome'] = non_syntenic
    transloc_chains = fetch_all_chains(chromosome, None, None, headers)
    if chromosome == 'chr2':  # don't count chr2A and chr2B as different
        transloc_chains = [c for c in transloc_chains if c.qName not in ['chr2A', 'chr2B']]
    translocations = sum([e.size for chain in transloc_chains for e in chain.entries if chain.qName != chromosome])
    chain_stats[chromosome]['Translocation from Other Chromosomes'] =  translocations
    chain_stats[chromosome]['First Three Chains'] = sum([e.size for chain in relevant[:3] for e in chain.entries])

chr2 chr2B None : 478
chr2 chr2A None : 798
chr1 chr1 None : 1168
chr1 None None : 1710
chr2 chr2 None : 0
chr2 None None : 1610
chr3 chr3 None : 473
chr3 None None : 661
chr4 chr4 None : 859
chr4 None None : 1126
chr5 chr5 None : 811
chr5 None None : 1206
chr6 chr6 None : 797
chr6 None None : 966
chr7 chr7 None : 1262
chr7 None None : 1643
chr8 chr8 None : 927
chr8 None None : 1135
chr9 chr9 None : 588
chr9 None None : 996
chr10 chr10 None : 929
chr10 None None : 1209
chr11 chr11 None : 554
chr11 None None : 701
chr12 chr12 None : 613
chr12 None None : 719
chr13 chr13 None : 529
chr13 None None : 701
chr14 chr14 None : 477
chr14 None None : 631
chr15 chr15 None : 263
chr15 None None : 461
chr16 chr16 None : 752
chr16 None None : 1105
chr17 chr17 None : 748
chr17 None None : 1003
chr18 chr18 None : 586
chr18 None None : 677
chr19 chr19 None : 818
chr19 None None : 986
chr20 chr20 None : 505
chr20 None None : 616
chr21 chr21 None : 298
chr21 None None : 541
chr22 chr22 None : 530
chr22 

In [64]:
import pandas as pd 
df = pd.DataFrame(chain_stats) 

In [65]:
df.head()

,chr1,chr10,chr11,chr12,chr13,chr14,chr15,chr16,chr17,chr18,...,chr22,chr3,chr4,chr5,chr6,chr7,chr8,chr9,chrX,chrY
First Three Chains,212843994,124487884,125843504,105788978,89790006,85221470,75656180,64139961,61506966,72710459,...,31328200,191117095,149671593,95936882,164380573,144691258,126838817,107005608,109515825,9056820
"Non-syntenic, Same Chromosome",104081729,39823372,78975113,63938230,31162967,513209,10205552,34476637,41881749,14511023,...,5221818,89029377,79986252,112038574,56884384,59326241,80149115,60177384,95470175,11040058
Primary Chain,114984699,86709438,48114280,63230073,62370844,85021656,69081961,38655719,33475161,58288802,...,27760708,102292762,102404731,44644988,107751895,90353609,58120074,50569308,48866364,4507514
Translocation from Other Chromosomes,3834877,1315050,571079,470757,745723,924394,1200080,4884563,1417764,635990,...,2766733,625415,1545241,17266605,1045473,2340038,1021778,5328211,744372,7456790


In [66]:
df['chr2']

First Three Chains                      211287080
Non-syntenic, Same Chromosome           107244972
Primary Chain                           124837690
Translocation from Other Chromosomes      1490781
Name: chr2, dtype: int64

In [34]:
df.to_csv(r"D:\Genomes\Human\alignments\hg38ToPanTro6_stats.csv")

In [42]:
chain_stats['chr5']

{'Non-syntenic, Same Chromosome': 112038574,
 'Primary Chain': 44644988,
 'Translocation from Other Chromosomes': 17266605}

In [44]:
c5 = fetch_all_chains('chr5', 'chr5', None, headers)
[str(c) for c in c5[:10]]

chr5 chr5 None : 811


['chain 3886028996 chr5 181538259 + 54020836 139452659 chr5 159319378 + 43603604 117629752 23',
 'chain 2334333576 chr5 181538259 + 18553211 46184988 chr5 159319378 - 85000998 112621309 38',
 'chain 2207293083 chr5 181538259 + 71209725 96585715 chr5 159319378 - 115719779 141012529 41',
 'chain 2176099837 chr5 181538259 + 155761324 181254502 chr5 159319378 + 133904970 159199629 43',
 'chain 1568613876 chr5 181538259 + 19928 18553204 chr5 159319378 + 465 18306847 48',
 'chain 1431413837 chr5 181538259 + 139453659 155760324 chr5 159319378 + 117630005 133905028 51',
 'chain 32852691 chr5 181538259 + 99410290 100047704 chr5 159319378 - 81110990 82035645 145',
 'chain 15380057 chr5 181538259 + 69789453 70364997 chr5 159319378 - 87719441 88683349 330',
 'chain 15182291 chr5 181538259 + 70129602 70358697 chr5 159319378 + 70788969 71612852 379',
 'chain 10279700 chr5 181538259 + 69643671 69785184 chr5 159319378 - 115715874 115850712 616']

# Comparison of Apes (PanTro5 and gorGor5)

In [32]:
all_results = {}
#TODO: include panTro5_to_Hg38
for comparison in 'hg38_to_panTro4 panTro4_to_Hg38 hg38_to_panTro5 hg38_to_gorGor4 gorGor4_to_Hg38 hg38_to_gorGor5 susie3_to_gorGor4'.split():
    all_results[comparison] = collect_stats(comparison)
    print()

hg38_to_panTro4 identity 98.65%
hg38_to_panTro4 coverage 93.26%

panTro4_to_Hg38 identity 98.68%
panTro4_to_Hg38 coverage 97.00%

hg38_to_panTro5 identity 98.65%
hg38_to_panTro5 coverage 95.44%

hg38_to_gorGor4 identity 98.29%
hg38_to_gorGor4 coverage 93.30%

gorGor4_to_Hg38 identity 98.34%
gorGor4_to_Hg38 coverage 96.15%

hg38_to_gorGor5 identity 98.32%
hg38_to_gorGor5 coverage 85.91%

susie3_to_gorGor4 identity 99.71%
susie3_to_gorGor4 coverage 96.72%



In [12]:
all_results['hg38_to_gorGor5']

{'hg38_to_gorGor5_chr1': {'Aligned Variance in bp': 3014826,
  'Alignment Coverage of Query Main Chr': 0.9967252562573473,
  'Alignment Coverage of Ref Chr': 0.8064999276984944,
  'Query Gaps larger than 1000bp': 1999,
  'Query Gaps larger than 100bp': 5310,
  'Query Gaps larger than 10bp': 23695,
  'Query Number of Gaps (all)': 367405,
  'Query unique bp': 610696,
  "Ref Chr Total Size (No N's)": 230472379,
  'Ref Gaps larger than 1000bp': 1642,
  'Ref Gaps larger than 100bp': 4977,
  'Ref Gaps larger than 10bp': 22362,
  'Ref Identity within Alignment': 0.9837804412756836,
  'Ref N to query bp': 18465410,
  'Ref Number of Gaps (all)': 219416,
  'Ref unique bp': 44596422,
  'Shared seq bp': 182861131,
  'Total alignment Length': 185875957},
 'hg38_to_gorGor5_chr10': {'Aligned Variance in bp': 1860982,
  'Alignment Coverage of Query Main Chr': 0.9963707325755958,
  'Alignment Coverage of Ref Chr': 0.8360626413211497,
  'Query Gaps larger than 1000bp': 682,
  'Query Gaps larger than 100